- Try a simple DNN for classification prediction
- Use the 64f data at first
- Also try with/without standardizing the data (maybe for NN the results are different)

In [11]:
# !pip install tensorflow
# !pip install scikeras

In [4]:
import tensorflow as tf
print(tf. __version__)

2.10.1


In [5]:
# Neural networks

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
# from tensorflow.keras.layers import Add, Input, Dense, Dropout, BatchNormalization, Embedding, Flatten, Concatenate, Reshape, Conv1D, AveragePooling1D, Multiply, MaxPool1D, Activation
from tensorflow.keras.regularizers import l1, l2
from scikeras.wrappers import KerasClassifier
from tensorflow_addons.layers import WeightNormalization

In [6]:
import pandas as pd 
import numpy as np
import sklearn
import pickle
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier, StackingClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, RandomizedSearchCV, cross_val_score

In [7]:
def make_submission(preds):
    assert len(preds) == 5000
    
    # Read labels
    with open('test_labels_sorted.npy', 'rb') as f:
        test_labels = np.load(f)
    len(test_labels)
    
    submission = pd.DataFrame(columns=['id', 'class'])
    for label, pred in zip(test_labels, preds):
        submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
    return submission

In [8]:
def plot_confusion_matrix(y_val_from_train, y_pred_from_train):
    fig, ax = plt.subplots(figsize=(8, 5))
    cmp = ConfusionMatrixDisplay(confusion_matrix(y_val_from_train, y_pred_from_train))
    cmp.plot(ax=ax)
    plt.show();

In [9]:
# Load X_train
with open('X_train_75_std.npy', 'rb') as f:
    X_train = np.load(f)
print(X_train.shape)
X_train

(9000, 75)


array([[ 0.66868409,  0.16353656,  0.63277468, ..., -0.68600237,
        -0.75071056, -0.6719989 ],
       [ 0.37202545,  0.27579237,  0.65904756, ...,  0.06479231,
        -0.48439794,  0.56611352],
       [-1.67047552, -1.2187742 , -2.37040642, ..., -0.65838631,
        -0.38095167, -0.1542973 ],
       ...,
       [-0.24270722, -0.14209669, -0.32285351, ...,  0.37591901,
        -0.37936549,  0.58761686],
       [ 0.38630325,  0.31266521,  0.37767149, ...,  0.83458681,
         0.3138927 ,  0.79569199],
       [ 0.33077964, -0.0822815 , -0.51143673, ..., -1.1650511 ,
        -0.410986  ,  0.44873132]])

In [10]:
# Load y_train
with open('y_train.npy', 'rb') as f:
    y_train = np.load(f)
print(y_train.shape)
y_train

(9000,)


array([ 7, 13,  6, ...,  3,  3, 18], dtype=int64)

In [11]:
# Load X_test
with open('X_test_75_std.npy', 'rb') as f:
    X_test = np.load(f)
print(X_test.shape)
X_test

(5000, 75)


array([[-0.59806336,  0.6264903 , -0.15256881, ...,  1.17376188,
         0.23880484,  0.48877589],
       [-0.47352949,  0.94441404, -0.23499654, ..., -0.41222519,
        -0.40989637, -0.65814035],
       [-0.64168924, -1.56209892, -1.06875674, ...,  0.3385773 ,
        -0.56342902,  0.74796695],
       ...,
       [ 1.00103725, -0.3371114 ,  0.37115316, ..., -1.16055904,
        -0.46010399, -0.62722221],
       [-0.98752639, -1.20156778, -1.41423994, ..., -0.81406819,
        -0.78249592,  1.12406601],
       [-0.21177287, -0.85742305, -0.54931118, ...,  0.02750228,
        -0.76809252, -1.25225668]])

## Make small changes to y_train

In [12]:
y_train -= 1

In [13]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=20)

In [14]:
print(y_train.shape)

(9000, 20)


## Let's Standard Scale X_train and X_test

In [127]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

In [128]:
# print(X_train)
# print()
# X_train = scaler.fit_transform(X_train)
# print(X_train)

[[-8.81665000e-01  1.04447100e+00  1.59010940e-01 ...  7.64136797e-02
   2.23461397e-02  1.29471645e+01]
 [-1.10552300e+00  1.12647200e+00  1.74547752e-01 ...  1.13979068e-01
   2.85805084e-02  1.38184886e+01]
 [-2.64679000e+00  3.47160000e-02 -1.61696002e+00 ...  7.77954261e-02
   3.10021816e-02  1.33114980e+01]
 ...
 [-1.56939900e+00  8.21211000e-01 -4.06112460e-01 ...  1.29546035e-01
   3.10393141e-02  1.38336216e+01]
 [-1.09474900e+00  1.15340700e+00  8.15228667e-03 ...  1.52495097e-01
   4.72684630e-02  1.39800549e+01]
 [-1.13664700e+00  8.64905000e-01 -5.17633651e-01 ...  5.24448737e-02
   3.02990790e-02  1.37358806e+01]]

[[ 0.66868409  0.16353656  0.63277468 ... -0.68600237 -0.75071056
  -0.6719989 ]
 [ 0.37202545  0.27579237  0.65904756 ...  0.06479231 -0.48439794
   0.56611352]
 [-1.67047552 -1.2187742  -2.37040642 ... -0.65838631 -0.38095167
  -0.1542973 ]
 ...
 [-0.24270722 -0.14209669 -0.32285351 ...  0.37591901 -0.37936549
   0.58761686]
 [ 0.38630325  0.31266521  0.37767

In [129]:
# print(X_test)
# print()
# X_test = scaler.transform(X_test)
# print(X_test)

[[-1.83755000e+00  1.38265100e+00 -3.05412347e-01 ...  1.69465441e-01
   4.55106589e-02  1.37640621e+01]
 [-1.74357700e+00  1.61488900e+00 -3.54157081e-01 ...  9.01118934e-02
   3.03245873e-02  1.29569175e+01]
 [-1.87047000e+00 -2.16077000e-01 -8.47212227e-01 ...  1.27677673e-01
   2.67303936e-02  1.39464683e+01]
 ...
 [-6.30872000e-01  6.78756000e-01  4.29758667e-03 ...  5.26696304e-02
   2.91492287e-02  1.29786762e+01]
 [-2.13143800e+00  4.72850000e-02 -1.05151830e+00 ...  7.00060125e-02
   2.16020454e-02  1.42111488e+01]
 [-1.54605600e+00  2.98677000e-01 -5.40031205e-01 ...  1.12113293e-01
   2.19392284e-02  1.25388069e+01]]

[[-0.59806336  0.6264903  -0.15256881 ...  1.17376188  0.23880484
   0.48877589]
 [-0.47352949  0.94441404 -0.23499654 ... -0.41222519 -0.40989637
  -0.65814035]
 [-0.64168924 -1.56209892 -1.06875674 ...  0.3385773  -0.56342902
   0.74796695]
 ...
 [ 1.00103725 -0.3371114   0.37115316 ... -1.16055904 -0.46010399
  -0.62722221]
 [-0.98752639 -1.20156778 -1.41423

In [130]:
# # Save X_train
# with open('X_train_75_std.npy', 'wb') as f:
#     np.save(f, X_train)
    
# with open('X_test_75_std.npy', 'wb') as f:
#     np.save(f, X_test)

## Neural Network

In [17]:
input_shape=75
depth=1
output_dim=20
batch_mod=2
num_neurons=16
drop_rate=0.1


def create_neural_network():
    inputs = tf.keras.layers.Input(shape=(input_shape,))
    batch1 = tf.keras.layers.BatchNormalization()(inputs)
    hidden1 = tf.keras.layers.Dense(num_neurons, activation='sigmoid')(batch1)
    dropout1 = tf.keras.layers.Dropout(drop_rate)(hidden1)
    hidden2 = tf.keras.layers.Dense(int(num_neurons/2), activation='sigmoid')(dropout1)
    
    added_layer = tf.keras.layers.Concatenate()([batch1] + [hidden2])
    [batch1].append(added_layer)
    b1 = tf.keras.layers.BatchNormalization()(added_layer)

    h1 = tf.keras.layers.Dense(num_neurons, activation='sigmoid')(b1)
    d1 = tf.keras.layers.Dropout(drop_rate)(h1)
    h2 = tf.keras.layers.Dense(int(num_neurons/2), activation='sigmoid')(d1)
    d2 = tf.keras.layers.Dropout(drop_rate)(h2)
    h3 = tf.keras.layers.Dense(int(num_neurons/2), activation='sigmoid')(d2)
    d3 = tf.keras.layers.Dropout(drop_rate)(h3)
    h4 = tf.keras.layers.Dense(int(num_neurons/2), activation='sigmoid')(d3)
    c1 = tf.keras.layers.Concatenate()([batch1] + [h4])
    output = tf.keras.layers.Dense(output_dim, activation='softmax')(c1)
    
    model = tf.keras.Model(inputs=inputs, outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

- Adam, epochs 300, bs 128, val_loss=0.18
- Adamax, epochs 300, bs 128, val_loss=0.19

In [18]:
estimator_neural_net = KerasClassifier(build_fn=create_neural_network)
estimator_neural_net.fit(X_train, y_train, validation_split=0.05, epochs=300, batch_size=128) # val loss 0.1864

Epoch 1/300


C:\Users\Sergiu\.conda\envs\tf\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


67/67 [==============================] - 1s 7ms/step - loss: 2.7409 - accuracy: 0.2127 - val_loss: 2.1930 - val_accuracy: 0.3156
Epoch 2/300
67/67 [==============================] - 0s 3ms/step - loss: 1.7370 - accuracy: 0.5053 - val_loss: 1.4926 - val_accuracy: 0.6133
Epoch 3/300
67/67 [==============================] - 0s 3ms/step - loss: 1.2447 - accuracy: 0.6937 - val_loss: 1.1288 - val_accuracy: 0.6956
Epoch 4/300
67/67 [==============================] - 0s 3ms/step - loss: 0.9700 - accuracy: 0.7751 - val_loss: 0.9213 - val_accuracy: 0.7578
Epoch 5/300
67/67 [==============================] - 0s 3ms/step - loss: 0.8069 - accuracy: 0.8147 - val_loss: 0.7953 - val_accuracy: 0.7978
Epoch 6/300
67/67 [==============================] - 0s 3ms/step - loss: 0.7027 - accuracy: 0.8303 - val_loss: 0.7061 - val_accuracy: 0.8089
Epoch 7/300
67/67 [==============================] - 0s 3ms/step - loss: 0.6348 - accuracy: 0.8415 - val_loss: 0.6417 - val_accuracy: 0.8156
Epoch 8/300
67/67 [======

KeyboardInterrupt: 

### Train on full dataset

In [138]:
estimator_neural_net = KerasClassifier(build_fn=create_neural_network)
estimator_neural_net.fit(X_train, y_train, epochs=300, batch_size=128)

Epoch 1/300


C:\Users\Sergiu\.conda\envs\tf\lib\site-packages\scikeras\wrappers.py:301: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  warnings.warn(


71/71 [==============================] - 1s 5ms/step - loss: 9.1228 - accuracy: 0.4063
Epoch 2/300
71/71 [==============================] - 0s 4ms/step - loss: 5.2757 - accuracy: 0.7449
Epoch 3/300
71/71 [==============================] - 0s 4ms/step - loss: 3.0690 - accuracy: 0.8177
Epoch 4/300
71/71 [==============================] - 0s 4ms/step - loss: 1.9015 - accuracy: 0.8434
Epoch 5/300
71/71 [==============================] - 0s 4ms/step - loss: 1.4508 - accuracy: 0.8619
Epoch 6/300
71/71 [==============================] - 0s 4ms/step - loss: 1.1753 - accuracy: 0.8701
Epoch 7/300
71/71 [==============================] - 0s 4ms/step - loss: 0.9980 - accuracy: 0.8801
Epoch 8/300
71/71 [==============================] - 0s 4ms/step - loss: 0.9086 - accuracy: 0.8859
Epoch 9/300
71/71 [==============================] - 0s 4ms/step - loss: 0.8589 - accuracy: 0.8896
Epoch 10/300
71/71 [==============================] - 0s 4ms/step - loss: 0.8259 - accuracy: 0.8914
Epoch 11/300
71/71 [=

71/71 [==============================] - 0s 4ms/step - loss: 0.1610 - accuracy: 0.9470
Epoch 165/300
71/71 [==============================] - 0s 4ms/step - loss: 0.1714 - accuracy: 0.9447
Epoch 166/300
71/71 [==============================] - 0s 4ms/step - loss: 0.1629 - accuracy: 0.9467
Epoch 167/300
71/71 [==============================] - 0s 4ms/step - loss: 0.1648 - accuracy: 0.9450
Epoch 168/300
71/71 [==============================] - 0s 4ms/step - loss: 0.1689 - accuracy: 0.9449
Epoch 169/300
71/71 [==============================] - 0s 4ms/step - loss: 0.1671 - accuracy: 0.9423
Epoch 170/300
71/71 [==============================] - 0s 4ms/step - loss: 0.1661 - accuracy: 0.9449
Epoch 171/300
71/71 [==============================] - 0s 4ms/step - loss: 0.1613 - accuracy: 0.9447
Epoch 172/300
71/71 [==============================] - 0s 4ms/step - loss: 0.1697 - accuracy: 0.9393
Epoch 173/300
71/71 [==============================] - 0s 4ms/step - loss: 0.1703 - accuracy: 0.9418
Epoc

KerasClassifier(
	model=None
	build_fn=<function create_neural_network_1 at 0x0000017C552E6550>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

In [139]:
preds = create_neural_network.predict(X_test)
preds

157/157 [==============================] - 0s 1ms/step


array([[0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [140]:
preds_as_array = np.argmax(preds, axis=1)
preds_as_array += 1
preds_as_array

array([3, 4, 5, ..., 4, 5, 1], dtype=int64)

In [141]:
sub = make_submission(preds_as_array)
sub

C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from p

C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from p

C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from p

C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from p

C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from p

C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from p

C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  submission = submission.append({'id': label, 'class': pred}, ignore_index=True)
C:\Users\Sergiu\AppData\Local\Temp\ipykernel_11408\2162223925.py:11: FutureWarning: The frame.append method is deprecated and will be removed from p

,id,class
0,10001,3
1,10002,4
2,10004,5
3,10008,20
4,10009,17
...,...,...
4995,23986,19
4996,23991,12
4997,23992,4
4998,23998,5


In [142]:
sub.to_csv('submission_keras_1depth_300e_128bs_64f_std.csv', index=False)